### Imports

In [1]:
# Standard imports
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
pio.kaleido.scope.mathjax = None


### Load scenarios

In [2]:
df_scenario_round_1 = pd.read_csv('scenarios/scenario_round_1.csv')

In [3]:
df_scenario_round_1

,Time (a.u.),Step,DA study critically needed,Optics,Optics link,Luminosity (1e34 cm2/s),Energy (TeV),ATS factor,Emittance (μm),Intensity (1e11 ppb),Beta (m),Octupoles (A),Crabs (μrad),Chromaticity
0,0.00,Start injection,N,No need to simulate,NaN,0,0.45,1.00,2.2,0,6,-10,0,15
1,0.20,End of injection,Y,opt_ramp_6000_thin (no phase knob!),https://gitlab.cern.ch/acc-models/acc-models-l...,0,0.45,1.00,2.2,2.3,6,-10,0,15
2,0.50,End of ramp,Y,opt_ramp_500_1500_thin,https://gitlab.cern.ch/acc-models/acc-models-l...,0,7.00,1.00,[2.2-2.5],2.3,0.5,-300,0,15
3,0.60,Start of collapse round,N,opt_collapse_1100_1500_thin,https://gitlab.cern.ch/iangelis/summer_optics/...,0,7.00,1.00,[2.2-2.5],2.3,1.1,-300,0,15
4,0.60,Start of collapse flat (alternative),N,opt_collapse_flathv_900_1800_1500_thin,https://gitlab.cern.ch/iangelis/summer_optics/...,0,7.00,1.00,[2.2-2.5],2.3,0.9/1.8,-300,0,15
5,0.65,End of collapse,Y,Same as previous step,Same as previous step,[2-2.5] (depends on filling scheme),7.00,1.00,[2.2-2.5],2.2,Same as previous step,-300,0,15
6,0.90,Start of levelling,Y,opt_levelling_580_1500_thin,https://gitlab.cern.ch/iangelis/summer_optics/...,5,7.00,1.00,[2.2-2.5],2.2,0.58,-60,optimized to reach target lumi/PU,15
7,2.00,End of levelling,Y,opt_round_150_1500_optphases_thin,https://gitlab.cern.ch/acc-models/acc-models-l...,5,7.00,3.33,[2.2-2.5],given by machine parameters,0.15,-60,190,15
8,2.20,Dump,N,No need to simulate,NaN,0,7.00,3.33,[2.2-2.5],unknown,0.15,-60,190,15


### Parse scenario

In [4]:
# Remove the row containing the "Start of collapse flat (alternative)" value in the step column
df_scenario_round_1 = df_scenario_round_1[df_scenario_round_1["Step"] != "Start of collapse flat (alternative)"]

# Reindex the dataframe
df_scenario_round_1 = df_scenario_round_1.reset_index(drop=True)

# Clean the float columns from string in the values
for idx, val in enumerate(df_scenario_round_1["Luminosity (1e34 cm2/s)"]):
    if isinstance(val, str) and "[" in val:
        print(val)
        df_scenario_round_1.loc[idx, "Luminosity (1e34 cm2/s)"] = np.mean(
            [float(x) for x in val.split("[")[1].split("]")[0].split("-")]
        )
df_scenario_round_1["Luminosity (1e34 cm2/s)"] = df_scenario_round_1["Luminosity (1e34 cm2/s)"].astype(float)

for idx, val in enumerate(df_scenario_round_1["Emittance (μm)"]):
    if isinstance(val, str):
        if "[" in val:
            df_scenario_round_1.loc[idx, "Emittance (μm)"] = np.mean(
                [float(x) for x in val.split("[")[1].split("]")[0].split("-")]
            )
        elif '(' in val:
            df_scenario_round_1.loc[idx, "Emittance (μm)"] = float(val.split(' (')[0])
df_scenario_round_1["Emittance (μm)"] = df_scenario_round_1["Emittance (μm)"].astype(float)

for idx, val in enumerate(df_scenario_round_1["Intensity (1e11 ppb)"]):
    if isinstance(val, str):
        if ' (' in val:
            df_scenario_round_1.loc[idx, "Intensity (1e11 ppb)"] = float(val.split(' (')[0])
        elif 'given' in val:
            df_scenario_round_1.loc[idx, "Intensity (1e11 ppb)"] = np.nan
        elif 'unknown' in val:
            df_scenario_round_1.loc[idx, "Intensity (1e11 ppb)"] = 1.4
df_scenario_round_1["Intensity (1e11 ppb)"] = df_scenario_round_1["Intensity (1e11 ppb)"].astype(float)
            
for idx, val in enumerate(df_scenario_round_1["Octupoles (A)"]):
    if isinstance(val, str) and "(" in val:
        df_scenario_round_1.loc[idx, "Octupoles (A)"] = float(val.split(' (')[0])
df_scenario_round_1["Octupoles (A)"] = df_scenario_round_1["Octupoles (A)"].astype(float)

for idx, val in enumerate(df_scenario_round_1["Crabs (μrad)"]):
    if isinstance(val, str) and "optimized" in val:
        df_scenario_round_1.loc[idx, "Crabs (μrad)"] = np.nan
df_scenario_round_1["Crabs (μrad)"] = df_scenario_round_1["Crabs (μrad)"].astype(float)

for idx, val in enumerate(df_scenario_round_1["Beta (m)"]):
    if isinstance(val, str) and "Same " in val:
        df_scenario_round_1.loc[idx, "Beta (m)"] = np.nan
df_scenario_round_1["Beta (m)"] = df_scenario_round_1["Beta (m)"].astype(float)



[2-2.5] (depends on filling scheme)


In [5]:
df_scenario_round_1

,Time (a.u.),Step,DA study critically needed,Optics,Optics link,Luminosity (1e34 cm2/s),Energy (TeV),ATS factor,Emittance (μm),Intensity (1e11 ppb),Beta (m),Octupoles (A),Crabs (μrad),Chromaticity
0,0.00,Start injection,N,No need to simulate,NaN,0.00,0.45,1.00,2.20,0.0,6.00,-10.0,0.0,15
1,0.20,End of injection,Y,opt_ramp_6000_thin (no phase knob!),https://gitlab.cern.ch/acc-models/acc-models-l...,0.00,0.45,1.00,2.20,2.3,6.00,-10.0,0.0,15
2,0.50,End of ramp,Y,opt_ramp_500_1500_thin,https://gitlab.cern.ch/acc-models/acc-models-l...,0.00,7.00,1.00,2.35,2.3,0.50,-300.0,0.0,15
3,0.60,Start of collapse round,N,opt_collapse_1100_1500_thin,https://gitlab.cern.ch/iangelis/summer_optics/...,0.00,7.00,1.00,2.35,2.3,1.10,-300.0,0.0,15
4,0.65,End of collapse,Y,Same as previous step,Same as previous step,2.25,7.00,1.00,2.35,2.2,NaN,-300.0,0.0,15
5,0.90,Start of levelling,Y,opt_levelling_580_1500_thin,https://gitlab.cern.ch/iangelis/summer_optics/...,5.00,7.00,1.00,2.35,2.2,0.58,-60.0,NaN,15
6,2.00,End of levelling,Y,opt_round_150_1500_optphases_thin,https://gitlab.cern.ch/acc-models/acc-models-l...,5.00,7.00,3.33,2.35,NaN,0.15,-60.0,190.0,15
7,2.20,Dump,N,No need to simulate,NaN,0.00,7.00,3.33,2.35,1.4,0.15,-60.0,190.0,15


### Plot scenarios

In [6]:
def plot_scenario(
    df_scenario,
    title_scenario,
    variables_to_plot=[
        "Energy (TeV)",
        "Emittance (μm)",
        "Intensity (1e11 ppb)",
        "Beta (m)",
        "Luminosity (1e34 cm2/s)",
        "Octupoles (1e2 A)",
        "Crabs (1e2 μrad)",
        "Chromaticity (/10)",
        "ATS factor",
    ],
    rescale_octupoles=True,
    rescale_crabs=True,
    rescale_chromaticity=True,
    save_to_pdf=True,
    save_to_png=True,
):

    # Rescale variables that are too large
    if rescale_octupoles and "Octupoles (A)" in df_scenario.columns:
        df_scenario["Octupoles (A)"] = df_scenario["Octupoles (A)"] / 100
        df_scenario = df_scenario.rename(columns={"Octupoles (A)": "Octupoles (1e2 A)"})
    if rescale_crabs and "Crabs (μrad)" in df_scenario.columns:
        df_scenario["Crabs (μrad)"] = df_scenario["Crabs (μrad)"] / 100
        df_scenario = df_scenario.rename(columns={"Crabs (μrad)": "Crabs (1e2 μrad)"})
    if rescale_chromaticity and "Chromaticity" in df_scenario.columns:
        df_scenario["Chromaticity"] = df_scenario["Chromaticity"] / 10
        df_scenario = df_scenario.rename(columns={"Chromaticity": "Chromaticity (/10)"})

    # Do the interactive plot
    fig = px.line(
        df_scenario,
        x="Time (a.u.)",
        y=variables_to_plot,
        markers=True,
        template="simple_white",
        color_discrete_sequence=px.colors.qualitative.Dark2,
        width=1000,
        height=8 * 00,
        hover_data=["Optics"],
    )

    # Improve plot aesthetics
    fig.update_traces(
        line={"width": 1.5},
        marker=dict(size=4),
        mode="markers+lines",
        hovertemplate=None,
        hoverlabel=dict(namelength=-1),
        connectgaps=True,
    )
    fig.update_yaxes(showgrid=False, title="Parameter value")
    fig.update_xaxes(
        showgrid=True,
        range=[df_scenario["Time (a.u.)"].iloc[0], df_scenario["Time (a.u.)"].iloc[-1]],
        title="Time (not to scale)",
        ticktext=df_scenario["Step"],
        tickvals=df_scenario["Time (a.u.)"],
    )
    fig.update_layout(
        legend_title="Parameters",
        font_family="Latin Modern Roman",
        title_font_family="Latin Modern Roman",
        title_text=title_scenario,
        title_x=0.5,
        hovermode="x unified",
    )

    # Show and possibly save the plot
    fig.show()
    if save_to_pdf:
        fig.write_image(f"plot_scenarios/{title_scenario.replace(' ', '_')}.pdf")
    if save_to_png:
        fig.write_image(f"plot_scenarios/{title_scenario.replace(' ', '_')}.png", scale=2)

In [7]:
plot_scenario(
    df_scenario=df_scenario_round_1,
    title_scenario="HL-LHC DA scenario 1 (round)",
)